In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from google.colab import files
uploaded = files.upload()
uploaded1 = files.upload()

Saving AppleStore.csv to AppleStore (1).csv


Saving appleStore_description.csv to appleStore_description (1).csv


In [4]:
import io 
AppleStore = pd.read_csv(io.BytesIO(uploaded['AppleStore.csv']))
reviews = pd.read_csv(io.BytesIO(uploaded1['appleStore_description.csv']))

In [5]:
Store = pd.merge(AppleStore, reviews, on='id')

In [6]:
Combine_AppleStore_df = Store.drop(columns=['track_name_y', 'size_bytes_y', 'rating_count_ver', 'user_rating_ver', 'vpp_lic', 'currency', 'cont_rating', 'ipadSc_urls.num'])

In [7]:
Combine_AppleStore_df = Combine_AppleStore_df.rename(columns = {'id': 'ID','track_name_x': 'TRACK NAME', 'size_bytes_x':'SIZE',
                                    'rating_count_tot':'TOTAL VOTES', 'user_rating':'RATING', 'ver':'VERSION', 'prime_genre': 'GENRE',  
                                    'sup_devices.num':'DEVICE SUPPORTED', 'lang.num':'LANGUAGE SUPPORTED', 'app_desc':'DESCRIPTION','price': 'PRICE'})

In [9]:
v=Combine_AppleStore_df['TOTAL VOTES']
R=Combine_AppleStore_df['RATING']
C=Combine_AppleStore_df['RATING'].mean()
m=Combine_AppleStore_df['TOTAL VOTES'].quantile(0.70)

In [10]:
Combine_AppleStore_df['WEIGHTED AVERAGE']=((R*v)+ (C*m))/(v+m)

In [11]:
AppleStore_sorted_ranking=Combine_AppleStore_df.sort_values('WEIGHTED AVERAGE',ascending=False)
AppleStore_sorted_ranking[['TRACK NAME', 'TOTAL VOTES', 'RATING', 'WEIGHTED AVERAGE', 'PRICE']]

,TRACK NAME,TOTAL VOTES,RATING,WEIGHTED AVERAGE,PRICE
1104,Head Soccer,481564,5.0,4.994726,0.00
302,Plants vs. Zombies,426463,5.0,4.994047,0.99
3086,Sniper 3D Assassin: Shoot to Kill Gun Game,386521,5.0,4.993435,0.00
2177,Geometry Dash Lite,370370,5.0,4.993150,0.00
498,Infinity Blade,326482,5.0,4.992234,0.99
...,...,...,...,...,...
4648,Philips Hue,1999,1.5,2.440485,0.00
942,Watch TNT,6856,2.0,2.307724,0.00
437,Blackboard Mobile Learn™,13567,2.0,2.172725,0.00
439,Hulu: Watch TV Shows & Stream the Latest Movies,56170,2.0,2.045634,0.00


In [12]:
AppleStore_sorted_ranking1=Combine_AppleStore_df.sort_values('TOTAL VOTES',ascending=False)
AppleStore_sorted_ranking1[['TRACK NAME', 'TOTAL VOTES', 'RATING', 'WEIGHTED AVERAGE', 'PRICE']]

,TRACK NAME,TOTAL VOTES,RATING,WEIGHTED AVERAGE,PRICE
16,Facebook,2974676,3.5,3.500016,0.00
519,Instagram,2161558,4.5,4.499222,0.00
1346,Clash of Clans,2130805,4.5,4.499210,0.00
707,Temple Run,1724546,4.5,4.499025,0.00
7,Pandora - Music & Radio,1126879,4.0,3.999275,0.00
...,...,...,...,...,...
4366,全国一斉性格テストDX,0,0.0,3.526956,0.00
4371,盗墓之王-寻龙秘事,0,0.0,3.526956,0.00
4373,Bibi & Tina: Pferdeabenteuer,0,0.0,3.526956,2.99
5788,S ou SS,0,0.0,3.526956,2.99


In [13]:
from sklearn.preprocessing import MinMaxScaler

scaling=MinMaxScaler()
AppleStore_scaled_df=scaling.fit_transform(Combine_AppleStore_df[['WEIGHTED AVERAGE','TOTAL VOTES']])
AppleStore_normalized_df=pd.DataFrame(Combine_AppleStore_df,columns=['WEIGHTED AVERAGE','TOTAL VOTES'])

In [14]:
Combine_AppleStore_df[['normalized_weight_average','normalized_total_votes']]= AppleStore_normalized_df

In [15]:
Combine_AppleStore_df['SCORE'] = Combine_AppleStore_df['normalized_weight_average'] * 0.5 + Combine_AppleStore_df['normalized_total_votes'] * 0.5
AppleStore_scored_df = Combine_AppleStore_df.sort_values(['SCORE'], ascending=False)
AppleStore_scored_df[['TRACK NAME', 'normalized_weight_average', 'normalized_total_votes', 'SCORE']]

,TRACK NAME,normalized_weight_average,normalized_total_votes,SCORE
16,Facebook,3.500016,2974676,1.487340e+06
519,Instagram,4.499222,2161558,1.080781e+06
1346,Clash of Clans,4.499210,2130805,1.065405e+06
707,Temple Run,4.499025,1724546,8.622752e+05
7,Pandora - Music & Radio,3.999275,1126879,5.634415e+05
...,...,...,...,...
4380,愛しのショコラティエ いけめん恋愛ゲーム・乙女げーむ女性向け人気,3.526956,0,1.763478e+00
5782,【完全無料】ソク出会える！ひみつフレ探し掲示板,3.526956,0,1.763478e+00
2786,花火撮影 notepad Lite,3.526956,0,1.763478e+00
2781,ATOK -日本語入力キーボード,3.526956,0,1.763478e+00


In [16]:
Combine_AppleStore_df.head(1)['DESCRIPTION']

0    SAVE 20%, now only $3.99 for a limited time!\n...
Name: DESCRIPTION, dtype: object

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

# Filling NaNs with empty string
Combine_AppleStore_df['DESCRIPTION'] = Combine_AppleStore_df['DESCRIPTION'].fillna('')

In [18]:
tfv_matrix = tfv.fit_transform(Combine_AppleStore_df['DESCRIPTION'])

In [19]:
from sklearn.metrics.pairwise import sigmoid_kernel

# Compute the sigmoid kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

In [20]:
indices = pd.Series(Combine_AppleStore_df.index, index=Combine_AppleStore_df['TRACK NAME']).drop_duplicates()

In [21]:
def get_rec(title, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[title]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar movies
    sig_scores = sig_scores[1:11]

    # Movie indices
    movie_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return Combine_AppleStore_df['TRACK NAME'].iloc[movie_indices]

In [22]:
get_rec('Instagram')

3749     Moments - private albums with friends and family
689       PhotoSync – transfer and backup photos & videos
787                                              PicFrame
4570                          InsPad - Instagram for iPad
3334    Video Saver & Uploader-Save & Upload your Phot...
1428    Wireless Transfer App:Sync backup photo and video
1245                                            Flipagram
998          Secret Calculator Pro - Hide photos & videos
206     Camera Plus: For Macro Photos & Remote Photogr...
3414    Google Photos - unlimited photo and video storage
Name: TRACK NAME, dtype: object